In [ ]:
import pandas as pd
import pyam
import nomenclature

In [ ]:
iamc_args = dict(
    model="State of CDR (2024) - CDR Gap",
    region="World",
    unit="Gt CO2/yr",
)

In [ ]:
dsd = nomenclature.DataStructureDefinition("../definitions/")

# Panel 9.2a

In [ ]:
def replace_stat(prefix, mapping, i):
    for key, value in mapping.items():
        i = i.replace(key, value)

    return f"{prefix} [{i}]"

In [ ]:
column_mapping = {
    "value": "Median",
    "max": "Maximum",
    "min": "Minimum"    
}


current_df = pyam.IamDataFrame(
    "source/9_gap/Ch9_fig2_gap_data.xlsx",
    sheet_name="panel_a_current",
    **iamc_args,
    scenario="Historical Reference",
    value=list(column_mapping),
)
current_df.rename(
    variable=dict([(i, replace_stat("Carbon Removal", column_mapping, i)) for i in current_df.variable]),
    inplace=True
)

In [ ]:
column_mapping = {
    "median": "Median",
    "percentile_5th": "p05",	
    "percentile_95th": "p95",	
    "percentile_25th": "p25",	
    "percentile_75th": "p75",	
    "max": "Maximum",
    "min": "Minimum"    
}

data = pd.read_excel("source/9_gap/Ch9_fig2_gap_data.xlsx", sheet_name="panel_a_scenario_range").drop(columns="var")

range_df = pyam.IamDataFrame(
    data,
    **iamc_args,
    scenario="Paris-Relevant Scenarios",
    value=list(column_mapping),
)
range_df.rename(
    variable=dict([(i, replace_stat("Carbon Removal", column_mapping, i)) for i in range_df.variable]),
    inplace=True,
)

In [ ]:
imp_df = pyam.IamDataFrame(
    "source/9_gap/Ch9_fig2_gap_data.xlsx",
    sheet_name="panel_a_imp_pathways",
    **iamc_args,
    scenario="imp",
    variable="Carbon Removal",
)

In [ ]:
type_df = pyam.IamDataFrame(
    "source/9_gap/Ch9_fig2_gap_data.xlsx",
    sheet_name="panel_b_bar_plots",
    usecols="C,E:G",
    **iamc_args,
    scenario="imp",
    variable="var"
)

type_df.rename(
    variable={
        "Novel CDR": "Carbon Removal|Novel", 
        "carbon_removal_land_direct": "Carbon Removal|Conventional"
    },
    inplace=True,
)

type_df.rename(
    scenario={
        "Current CDR": "Historical Reference", 
    },
    inplace=True,
)

In [ ]:
type_df.scenario

In [ ]:
df = pyam.concat([current_df, range_df, imp_df, type_df])

In [ ]:
df.plot()

In [ ]:
dsd.validate(df)

# Upload to database

In [ ]:
df.set_meta("Chapter 9", "Chapter")

In [ ]:
df.to_ixmp4("socdr-dev")